# Capstone Project
This project is mainly for the Capston Project

In [13]:
#importing necessary packages
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

#Web Scrappping with Beautiful Soup
page_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page=requests.get(page_link)
soup = BeautifulSoup(page.content, 'html.parser')
table_soup = soup.find_all('table')[0] 
table = pd.read_html(str(table_soup))

#Creating the table dataframe 
data_frame=pd.DataFrame(table[0])
data_frame.columns=['Postcode','Borough','Neighbourhood']
data_frame=data_frame.drop(0).reset_index(drop=True)

In [14]:
#Deleting the Not Assigned in the Borough Rows
data_frame['Borough']=data_frame['Borough'].replace(to_replace='Not assigned' , value=np.nan)
data_frame = data_frame[pd.notnull(data_frame['Borough'])].reset_index(drop=True)

In [15]:
#combining the rows of the same postacode value
postal={}
for index,row in data_frame.iterrows():
    row=dict(row)
    if row['Postcode'] in postal:
        
        postal[row['Postcode']]['Neighbourhood']=postal[row['Postcode']]['Neighbourhood']+', '+row['Neighbourhood']
        
    else:
        if row['Neighbourhood']=='Not assigned':
            row['Neighbourhood']=row['Borough']
        postal[row['Postcode']]=row 
df1=pd.DataFrame(postal).transpose()
df1.set_index('Postcode',inplace=True)
df1=df1.reset_index(drop=False)
df1.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [16]:
df1.shape[0]

103